# Script to test connection to database

In [ ]:
import pyodbc
import pandas as pd
import datetime as dt


def load_table_from_db(server, database, username, password, sql_query):
    txt = """DRIVER={ODBC Driver 17 for SQL Server};SERVER={server};
             DATABASE={db};UID={user};PWD={pwd}"""
    db_info = txt.format(server=server, db=database, user=username, pwd=password)
    cnxn = pyodbc.connect(db_info)
    cursor = cnxn.cursor()
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    cnxn.close()
    df = pd.DataFrame(rows)
    return df

def load_orders_from_db(server, database, username, password):
    sql_query = """
        select t3.Fefco_Teil, t3.ArtNr_Teil, t3.ID_Druck, t3.Bogen_Laenge_Brutto, 
            t3.Bogen_Breite_Brutto, t1.MaschNr, t1.Ruestzeit_Soll, t1.Laufzeit_Soll,
            t1.Zeit_Soll, t3.Werkzeug_Nutzen, t3.Bestell_Nutzen, t1.Menge_Soll, 
            t3.Bemerkung, t2.LTermin , t2.KndNr, t4.Suchname, t1.AKNR, t1.TeilNr, t1.SchrittNr, 
            t7.Lieferdatum as Lieferdatum_Rohmaterial, t7.BE_Erledigt, t5.Start, t5.Ende
        from tbl_Produktion_Fertigungsschritte t1 inner join
            tbl_Produktion t2 ON t1.AKNR = t2.AKNr inner join
            tbl_Produktion_Teil t3 ON t1.AKNR = t3.AKNR inner join
            tbl_Kunden t4 on t2.KndNr = t4.KndNr inner join
            tbl_Maschine_AK_Zeiten t5 on t1.AKNR = t5.AKNR and t1.TEILNR = t5.TEILNR and t1.ID_MaschNr = t5.MaschNr inner join
            tbl_Maschine_Status t6 on t5.ID_Maschstatus = t6.ID_Maschstatus left outer join
            tbl_Bestellung_Kopf t7 on t3.BESTNR = t7.BENR
        where SchrittNr <> 0 and ID_MaschNr <> 1 and Start is NULL or Start >= {today}""".format(today=dt.datetime.now().strftime('%Y-%m-%d'))
    df = load_table_from_db(server, database, username, password, sql_query)
    return df

def load_werksplan_from_db(server, database, username, password, start_date, end_date):
    sql_query = """
        select tbl_Produktion_Planung.DATUM, tbl_Produktion_Planung.JAHR, tbl_Produktion_Planung.MONAT, tbl_Produktion_Planung.KW, 
            tbl_Produktion_Planung.TAG, tbl_Produktion_Planung.WOCHENTAG, tbl_Produktion_Planung.START, tbl_Produktion_Planung.ENDE, 
            tbl_Produktion_Planung.P1_START, tbl_Produktion_Planung.P1_ENDE, tbl_Produktion_Planung.P2_START, tbl_Produktion_Planung.P2_ENDE, 
            tbl_Produktion_Planung.P3_START, tbl_Produktion_Planung.P3_ENDE, tbl_Produktion_Planung.ARBEITSZEIT_MIN, tbl_Produktion_Planung.FEIERTAG
        from schulte.dbo.tbl_Produktion_Planung tbl_Produktion_Planung
        where DATUM >= {start} and DATUM <= {end}""".format(start=start_date, end=end_date)
    df = load_table_from_db(server, database, username, password, sql_query)
    return df

def load_maschinenplanung_from_db(server, database, username, password, start_date, end_date):
    sql_query = """
        select tbl_Produktion_Planung_Maschine.DATUM, tbl_Produktion_Planung_Maschine.ID_MASCHINE, tbl_Produktion_Planung_Maschine.MSTART, 
            tbl_Produktion_Planung_Maschine.MENDE, tbl_Produktion_Planung_Maschine.MP1_START, tbl_Produktion_Planung_Maschine.MP1_ENDE, 
            tbl_Produktion_Planung_Maschine.MP2_START, tbl_Produktion_Planung_Maschine.MP2_ENDE, tbl_Produktion_Planung_Maschine.MP3_START, 
            tbl_Produktion_Planung_Maschine.MP3_ENDE, tbl_Produktion_Planung_Maschine.MZEIT_MIN
        from schulte.dbo.tbl_Produktion_Planung_Maschine tbl_Produktion_Planung_Maschine
        where DATUM >= {start} and DATUM <= {end}""".format(start=start_date, end=end_date)
    df = load_table_from_db(server, database, username, password, sql_query)
    return df

In [ ]:
server =
database =
username = 
password = 
df_order = load_orders_from_db(server, database, username, password)
df_order

In [ ]:
start_date = '04.04.2022 00:00:00'
end_date = '08.04.2022 00:00:00'
df_werksplan = load_werksplan_from_db(server, database, username, password, start_date, end_date)
df_werksplan

In [ ]:
df_maschinenplan = load_maschinenplanung_from_db(server, database, username, password, start_date, end_date)
df_maschinenplan